# MORDM

In [2]:
# Import general python packages
import pandas as pd
import numpy as np
import copy

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over,time_step_0,time_step_1, time_step_2, time_step_3, time_step_4

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome, TimeSeriesOutcome, ArrayOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [3]:
# Loading in all the 17 objectives via predefined problem formulation 3
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)
    dike_model, planning_steps = get_model_for_problem_formulation(7)

In [4]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(outcome)
    

ArrayOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over_time at 0x0000019E65BC39C0>)
ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('Total_period_Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs', 'A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over_time at 0x0000019E65BC39C0>)


In [5]:
values = dike_model.outcomes['Total_period_Costs'].variable_name
print(len(values))  # This will show you the number of elements

12


In [6]:
# with MultiprocessingEvaluator(dike_model) as evaluator:
#     results = evaluator.perform_experiments(scenarios= 1, policies=1)

In [7]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(outcome.shape)

None
None
None
None
None
None
None


In [17]:
# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    # cost_variables.extend(
    # [
    #     f"{dike}_{e}"
    #     for e in ["Expected Annual Damage", "Dike Investment Costs"]
    #     for dike in function.dikelist
    # ])
    # cost_variables.extend([f"RfR Total Costs"])
    # cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)]
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A2', variable_name='A.2_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A3', variable_name='A.3_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A1', variable_name='A.1_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A2', variable_name='A.2_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A3', variable_name='A.3_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A4-A5', variable_name = 
            ['A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'], function = sum_over, kind=direction),
            ]

    elif problem_formulation_actor == 6: #OVERIJSSEL
        model.outcomes.clear()
        
        model.outcomes = [
            ScalarOutcome(f'Total_period_Costs 0', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_0, kind=direction),
            ScalarOutcome(f'Total_period_Costs 1', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_1, kind=direction),
            ScalarOutcome(f'Total_period_Costs 2', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_2, kind=direction),
            ScalarOutcome(f'Total_period_Costs 3', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_3, kind=direction),
            ScalarOutcome(f'Total_period_Costs 4', variable_name= dike_model.outcomes['Total_period_Costs'].variable_name, function = time_step_4, kind=direction),
            ScalarOutcome("Expected Number of Deaths_", variable_name =
            [f"{dike}_Expected Number of Deaths_" for dike in function.dikelist], function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A4_', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A5_', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction)
        ]
    else:
        raise TypeError('unknown identifier')
        
    return model

In [18]:
# Replicate the uncertainties
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [19]:
levers = dike_model.levers 
levers = copy.deepcopy(dike_model.levers)

# Overijssel

In [20]:
model = problem_formulation_actor(6)

In [21]:
for tr in model.outcomes:
    print(tr)

ScalarOutcome('Total_period_Costs 0', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs', 'A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function time_step_0 at 0x0000019E65BD80E0>)
ScalarOutcome('Total_period_Costs 1', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs', 'A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function time_step_1 at 0x0000019E65BD8180>)
ScalarOutcome('Total_period_Costs 2', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Inve

In [22]:
reference_values = {
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.5,
    "ID flood wave shape": 4,
    "planning steps": 2,
}

reference_values.update({f"discount rate {n}": 3.5 for n in planning_steps})
refcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        refcase_scen.update({key.name: reference_values[key.name]})
    else:
        refcase_scen.update({key.name: reference_values[name_split[1]]})
            
ref_scenario = Scenario('reference', **refcase_scen)

In [39]:
def constraint_function(*arg):
    threshold = 20000
    return max(0, arg[0] - threshold)

In [40]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total_period_Costs 0", outcome_names="Total_period_Costs 0", function=constraint_function)]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(1):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                    constraints = constraint)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 4 workers

  0%|                                                    | 0/2 [00:00<?, ?it/s]
100it [00:46,  2.14it/s]                                                       
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool


KeyError: 'epsilon_progress'

In [41]:
y,t = results

In [42]:
y

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.5_DikeIncrease 3,A.5_DikeIncrease 4,Total_period_Costs 0,Total_period_Costs 1,Total_period_Costs 2,Total_period_Costs 3,Total_period_Costs 4,Expected Number of Deaths_,Expected Annual Damage A4_,Expected Annual Damage A5_
0,0,0,1,0,0,0,0,1,0,0,...,2,9,1.119030e+08,2.366773e+08,6.407092e+08,2.465557e+08,5.152935e+08,0,0.0,0.0


In [26]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total_costs_0", outcome_names="Total_period_Costs_0", function=lambda x: max(0, x - 1000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(1):
       yui = evaluator.optimize(nfe=3, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                constraints=)
        
        



[MainProcess/INFO] pool started with 4 workers
100it [00:50,  1.98it/s]                                                       
[MainProcess/INFO] optimization completed, found 41 solutions
[MainProcess/INFO] terminating pool


In [28]:
y,t = yui

In [29]:
y

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.5_DikeIncrease 3,A.5_DikeIncrease 4,Total_period_Costs 0,Total_period_Costs 1,Total_period_Costs 2,Total_period_Costs 3,Total_period_Costs 4,Expected Number of Deaths_,Expected Annual Damage A4_,Expected Annual Damage A5_
0,1,0,0,1,1,0,1,1,0,0,...,7,10,2.689981e+08,8.389240e+08,5.333821e+08,5.957357e+08,4.773564e+08,0,0.0,0.0
1,0,0,0,0,1,0,1,1,1,1,...,2,0,2.880664e+08,8.382729e+08,4.711590e+08,4.847217e+08,8.219459e+08,0,0.0,0.0
2,1,0,1,0,1,1,1,0,0,0,...,5,10,3.407255e+08,5.504219e+08,5.223962e+08,7.547295e+08,8.075654e+08,0,0.0,0.0
3,1,1,0,0,0,0,0,1,1,0,...,2,4,6.298802e+08,3.396718e+08,5.297190e+08,7.676525e+08,7.327391e+08,0,0.0,0.0
4,0,1,0,0,0,1,0,0,1,0,...,2,0,4.989633e+08,3.537337e+08,3.574981e+08,4.623613e+08,2.801800e+08,0,0.0,0.0
5,0,1,1,0,1,1,1,0,1,0,...,9,6,5.283797e+08,9.297634e+08,2.655742e+08,5.896235e+08,6.344942e+08,0,0.0,0.0
6,0,1,0,0,1,0,0,1,0,0,...,1,0,4.764915e+08,3.526661e+08,5.760782e+08,5.370849e+08,6.622838e+08,0,0.0,0.0
7,1,1,1,0,1,0,0,0,1,1,...,3,9,5.501676e+08,3.072907e+08,3.637280e+08,8.794727e+08,5.810658e+08,0,0.0,0.0
8,1,0,0,0,1,1,0,1,0,0,...,2,10,5.027145e+08,2.273453e+08,6.357634e+08,5.359464e+08,4.103867e+08,0,0.0,0.0
9,1,1,1,1,1,0,0,0,0,0,...,3,3,4.117041e+08,3.797661e+08,5.295695e+08,4.843347e+08,6.363449e+08,0,0.0,0.0


In [ ]:
from ema_workbench.em_framework.optimization import epsilon_nondominated, to_problem

problem = to_problem(model, searchover="levers")
epsilons = [0.05] * len(model.outcomes)
merged_archives = epsilon_nondominated(results_df, epsilons, problem)

In [ ]:
(y1,t1) , (y2,t2) = results

In [ ]:
t2

In [ ]:
y = y1+y2
y

# Gelderland


In [ ]:
model = problem_formulation_actor(5)
for outcome in model.outcomes:
    print(repr((outcome)))

In [ ]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 700000000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(2):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                     constraints=constraint)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')
